## Name scope

Group nodes together with tf.name_scope(name)

```python
with tf.name_scope('data'):
    iterator = dataset.make_initializable_iterator()
    center_words, target_words = iterator.get_next()
```


## Variable scope

Name scope vs variabel scope

tf.name_scope() vs tf.variable_scope()

**Variable scope faciliatate variable sharing**

그냥 tf.Variable을 쓰면 별개의 Variable이 생긴다. -> tf.get_variable()을 사용해야 한다.

### tf.get_variable()

If a variable with name already exists, reuse it. If not, initialize it with shape using initializer

In [13]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

In [14]:
def two_hidden_layers(x):
    #assert x.shape.as_list() == [200,100]
    w1 = tf.get_variable("h1_weights",  [100,50], initializer=tf.random_normal_initializer())
    b1 = tf.get_variable("h1_baises", [50], initializer=tf.constant_initializer(0.0))
    h1 = tf.matmul(x, w1) + b1
    
    assert h1.shape.as_list() == [200, 50]
    w2 = tf.get_variable("h2_weights", [50, 10], initializer=tf.random_normal_initializer())
    b2 = tf.get_variable("h2_baises", [10], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(h1, w2) + b2
    
    return logits

In [15]:
x1 = tf.truncated_normal([200,100], name='x1')

logits1 = two_hidden_layers(x1)


In [16]:
x2 = tf.truncated_normal([200,100], name='x2')

logits2 = two_hidden_layers(x2)

ValueError: Variable h1_weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\Program Files\Anaconda3\envs\tensorflow_1_7\lib\site-packages\tensorflow\python\framework\ops.py", line 1654, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "D:\Program Files\Anaconda3\envs\tensorflow_1_7\lib\site-packages\tensorflow\python\framework\ops.py", line 3290, in create_op
    op_def=op_def)
  File "D:\Program Files\Anaconda3\envs\tensorflow_1_7\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


**tf.variable_scope implicitly creates a name scope**

In [17]:
with tf.variable_scope('two_layers') as scope:
    logits1 = two_hidden_layers(x1)
    scope.reuse_variables()
    logtis2 = two_hidden_layers(x2)

### Layer 'em up

In [18]:
def fully_connected(x, output_dim, scope):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        w = tf.get_variable("weights", [x.shape[1], output_dim], initializer=tf.random_normal_initializer())
        b = tf.get_variable("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        return tf.matmul(x,w)+b
    
    def two_hidden_layer(x):
        h1 = fully_connected(x, 50, 'h1')
        h2 = fully_connected(h1, 10, 'h2')
        
    with tf.variable_scope('two_layers') as scope:
        logits1 = two_hidden_layers(x1)
        logits2 = two_hidden_layer(x2)

## Manage Experiments

## tf.train.Saver

saves graph's variables in binary files

A good practice is to periodically save the model's parameters after a certain number of steps so that we can restore/retrain our model from that step if need be. 

**tf.train.Saver.save(sess, save_path, global_step=None...)**

**tf.train.Saver.restore(sess, save_path)**

In [1]:
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import utils

DATA_FILE = './birth_life_2010.txt'

data, n_samples = utils.read_birth_life_data(DATA_FILE)

X= tf.placeholder(tf.float32, shape=None, name='x')
Y = tf.placeholder(tf.float32, shape=None, name='y')

w = tf.get_variable(name='w', dtype=None, initializer=tf.constant(0.0))
b = tf.get_variable(name='b', dtype=None, initializer=tf.constant(0.0))

Y_predicted = w*X + b

loss = tf.square(Y - Y_predicted, name='loss')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Step 8 : train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x, y in data:
            # Execute train_op and get the value of loss.
            _, loss_ = sess.run([optimizer, loss], feed_dict={X: x, Y: y})
            total_loss += loss_
            
        print('Epoch {0}: {1}'.format(i, total_loss/n_samples))
        if i % 10 == 0:
            saver.save(sess, 'checkpoint_directory/model_name', global_step=i)
    # Step 9 :output the values of w and b
    w_out, b_out = sess.run([w, b])
    

Epoch 0: 1661.8637834631543
Epoch 1: 956.3224148609137
Epoch 2: 844.6737023980994
Epoch 3: 750.7312486011339
Epoch 4: 667.6598341012079
Epoch 5: 594.1417715627896
Epoch 6: 529.07878103068
Epoch 7: 471.5004191489204
Epoch 8: 420.5458626462441
Epoch 9: 375.45530721966765
Epoch 10: 335.5543025185697
Epoch 11: 300.24629857978107
Epoch 12: 269.00376475843336
Epoch 13: 241.35957466852116
Epoch 14: 216.90039135300015
Epoch 15: 195.25972298129324
Epoch 16: 176.1137693605349
Epoch 17: 159.17551693441837
Epoch 18: 144.1907111125557
Epoch 19: 130.93503488078713
Epoch 20: 119.20935661137888
Epoch 21: 108.8379309807855
Epoch 22: 99.66466760624593
Epoch 23: 91.55177013029001
Epoch 24: 84.37664046781751
Epoch 25: 78.03217824997724
Epoch 26: 72.42182927812989
Epoch 27: 67.46136239485718
Epoch 28: 63.07566952367442
Epoch 29: 59.19874146522856
Epoch 30: 55.77168446383194
Epoch 31: 52.74269822355127
Epoch 32: 50.065632780875376
Epoch 33: 47.70006421631674
Epoch 34: 45.61017902122909
Epoch 35: 43.76379750

it's helpful to append the number of training steps our model has gone through in a variable called global_step. It's very common variable to see in TF program. 

We first need to create it, initialize it to 0 and set it to be not trainable, since we dont want to TF to optimize it.

```python
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

optimizer = tf.train.Adamoptimizer(lr).minimize(loss, global_step=global_step)
```

Need to tell optimizer to increment global step

This can also help your optimizer know when to decay learning rate.

Only save variables, not graph

Checkpoints map variable names to tensors

```python
v1 = tf.Variable(name='v1')
v2 = tf.Variable(name='v2')

saver = tf.train.Saver({'v1':v1, 'v2':v2})
saver = tf.train.Saver([v1, v2])
saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})
```

### Restore variables

saver.restore(sess, 'checkpoints/name_of_the_checkpoint')


### Restore the latest checkpoint


checkpoint file keeps track of the latest checkpoint

restore checkpoints only when there is a valid checkpoint path

```python
# check if there is checkpoint
ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))

# check if there is a valid checkpoint path
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
```


## tf.summary

Visualize our summary statisitcs during our training

**tf.summary.scalar**

**tf.summary.histogram**

**tf.summary.image**

```python
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", self.loss)
    tf.summary.scalar("accuracy", self.accuracy)
    tf.summary.histogram("histogram loss", self.loss)
    # merge them all into one summary op to make managing them easier
    summary_op = tf.summary.merge_all()
```

summaries are ops, for the summaries to be built, you have to run it in a session.
```python
loss_batch, _, summary = sess.run([loss, optimizer, summary_op])
```

need global step here so the model knows what summary corresponds to what step
```python
writer.add_summary(summary, global_step=step)
```

In [5]:
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import utils

DATA_FILE = './birth_life_2010.txt'

tf.reset_default_graph()

data, n_samples = utils.read_birth_life_data(DATA_FILE)

X= tf.placeholder(tf.float32, shape=None, name='x')
Y = tf.placeholder(tf.float32, shape=None, name='y')

w = tf.get_variable(name='w', dtype=None, initializer=tf.constant(0.0))
b = tf.get_variable(name='b', dtype=None, initializer=tf.constant(0.0))

Y_predicted = w*X + b

loss = tf.square(Y - Y_predicted, name='loss')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

tf.summary.scalar("loss", loss)
tf.summary.histogram("histogram loss", loss)
summary_op = tf.summary.merge_all()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./board/linear/lr0.5', sess.graph)
    # Step 8 : train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x, y in data:
            # Execute train_op and get the value of loss.
            _, loss_, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: x, Y: y})
            total_loss += loss_
        writer.add_summary(summary, global_step=i)
        print('Epoch {0}: {1}'.format(i, total_loss/n_samples))
        if i % 10 == 0:
            saver.save(sess, 'checkpoint_directory/model_name', global_step=i)
    # Step 9 :output the values of w and b
    w_out, b_out = sess.run([w, b])
    writer.close()
    

INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
Epoch 0: 1661.8637834631543
Epoch 1: 956.3224148609137
Epoch 2: 844.6737023980994
Epoch 3: 750.7312486011339
Epoch 4: 667.6598341012079
Epoch 5: 594.1417715627896
Epoch 6: 529.07878103068
Epoch 7: 471.5004191489204
Epoch 8: 420.5458626462441
Epoch 9: 375.45530721966765
Epoch 10: 335.5543025185697
Epoch 11: 300.24629857978107
Epoch 12: 269.00376475843336
Epoch 13: 241.35957466852116
Epoch 14: 216.90039135300015
Epoch 15: 195.25972298129324
Epoch 16: 176.1137693605349
Epoch 17: 159.17551693441837
Epoch 18: 144.1907111125557
Epoch 19: 130.93503488078713
Epoch 20: 119.20935661137888
Epoch 21: 108.8379309807855
Epoch 22: 99.66466760624593
Epoch 23: 91.55177013029001
Epoch 24: 84.37664046781751
Epoch 25: 78.03217824997724
Epoch 26: 72.42182927812989
Epoch 27: 67.46136239485718
Epoch 28: 63.07566952367442
Epoch 29: 59.19874146522856
Epoch 30: 55.77168446383194
Epoch 31: 52.74269822355127
Epoch 32: 50.0656327

## Control Randomization

### Session keep track of random state

Each new session restarts the random state.

In [6]:
c = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c))
    print(sess.run(c))

3.574932
-5.9731865


In [7]:
c = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c))
    
with tf.Session() as sess:
    print(sess.run(c))

3.574932
3.574932


### op level random seed

Each op keeps its own seed

In [8]:
c = tf.random_uniform([], -10, 10, seed=2)
d = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c))
    print(sess.run(d))

3.574932
3.574932


### Graph level seed

In [15]:
tf.set_random_seed(2)
c = tf.random_uniform([], -10, 10)
d = tf.random_uniform([], -10, 10)

with tf.Session() as sess:
    print(sess.run(c))
    print(sess.run(d))

-1.1094971
8.42697


## Autodiff

Tensorflow builds the backward path for you.

### tf.gradients(y, [xs])

Take derivative of y with respect to each tensor in the list[xs]

In [16]:
x = tf.Variable(2.0)
y = 2.0 * (x**3)
z = 3.0 + y **2

grad_z = tf.gradients(z, [x,y])

with tf.Session() as sess:
    sess.run(x.initializer)
    print(sess.run(grad_z))

[768.0, 32.0]
